# Analytics

## Unigrams/Bigrams summary

Parse and convert all data to a Pandas Dataframe

In [16]:
import pandas as pd
import json
from pathlib import Path

In [17]:
base_path = Path("semeval2017-task8-dataset/rumoureval-data")
sentiment_file = Path("./semeval2017-task8-dataset/traindev/rumoureval-subtaskA-train.json")

with open(sentiment_file, 'r') as f:
    sentiment_dict = json.load(f)

records = []

for topic_dir in base_path.iterdir():
    topic = topic_dir.name

    for tweet_dir in topic_dir.iterdir():
        replies_dir = tweet_dir / "replies"

        for json_file in replies_dir.glob("*.json"):
            with open(json_file, 'r', encoding='utf-8') as f:
                tweet_data = json.load(f)
            
            tweet_id = str(tweet_data.get('id', ''))
            record = {
                'tweet_id': tweet_id,
                'topic': topic,
                'text': tweet_data.get('text', ''),
                'sentiment': sentiment_dict.get(tweet_id)
            }
            records.append(record)

df = pd.DataFrame(records)

In [19]:
# Display basic information
print(f"Total tweets: {len(df)}")
print(f"DataFrame shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

# Show a few samples
print("\nFirst 5 rows:")
print(df.head())

# Show sentiment distribution
if 'sentiment' in df.columns:
    print("\nSentiment distribution:")
    print(df['sentiment'].value_counts(dropna=False))
    print(f"\nTweets with sentiment: {df['sentiment'].notna().sum()}")
    print(f"Tweets without sentiment: {df['sentiment'].isna().sum()}")

# Show tweet type distribution
if 'type' in df.columns:
    print("\nTweet type distribution:")
    print(df['type'].value_counts())

Total tweets: 4222
DataFrame shape: (4222, 4)
Columns: ['tweet_id', 'topic', 'text', 'sentiment']

First 5 rows:
             tweet_id         topic  \
0  553549946738446336  charliehebdo   
1  553547724860456961  charliehebdo   
2  553551580801536000  charliehebdo   
3  553553099470946304  charliehebdo   
4  553544770430464000  charliehebdo   

                                                text sentiment  
0  @PD28 @BklynMiddleton Hashtag #kosher shows lo...     query  
1                                   *Mutes Universe*   comment  
2  @BklynMiddleton rather than being a 'lone-wolf...     query  
3  @PD28 @BklynMiddleton No problem! Wish it were...   comment  
4  RT @BklynMiddleton: An 11-month old baby is am...   support  

Sentiment distribution:
sentiment
comment    2724
support     587
query       330
deny        325
None        256
Name: count, dtype: int64

Tweets with sentiment: 3966
Tweets without sentiment: 256


## LDA

# Classification

## 4-way Classification

## Model Prompting

## Chain of thought